# C++的类

C++中的类来源于C中结构体的使用，类除了能像C中的结构体那样封装变量之外还可以封装函数。虽然C++将结构体的功能也进行了这样的拓展，使得这两个概念的用法极为相似，但实际开发中会应用面向对象的编程思想，因此在实际使用中要采用这样的思想，最好还是使用类。

类和结构体相似，在对其进行定义的时候，只是给出了其抽象的描述，描述了其成员变量和成员函数，而没有给出其具体的实现，其具体的实现要靠创建类的对象来完成

类的概念的引入要考虑以下几个问题：
1. 如何定义一个类？其成员变量如何声明，其成员函数如何声明？
2. 类在创建的时候，其成员变量没有初值，如何初始化？
3. 面向对象的三大特性之一的封装，类如何实现封装？

## 创建类

In [2]:
#include<iostream>
#include<string>
using namespace std;
class Student
{
    // 私有成员    
	private:
		string m_name;  //字符串类型数据，C++拓展出的内容
		int m_age;
		float m_score;
    // 公有成员
    public:
        Student();  //重载的构造函数
        Student(string name, int age, float score);  //构造函数的声明
		void say();  //成员函数,一般只给声明在这里 
};
void Student::say()
{
    cout << m_name << "的年龄是" << m_age << "成绩是"<< m_score << endl; 
} 

函数成员一般加一个“m_”前缀，以区别其他的变量名

为了达到封装的目的，一般将成员的权限限制为private，把成员函数设置成public，让外部函数只能通过成员函数来访问成员变量

一般在类内除了内联函数，只声明函数不给出函数的具体定义，将函数的具体定义写在类外。

## 类所对应的对象如何创建？

### 构造函数

In [3]:
Student::Student(string name, int age, float score)
{
    m_name = name;
    m_age = age;
    m_score = score;
}

构造函数的作用就是在创建对象时，对类内的各成员函数赋初值。
构造函数只要程序员定义了就一定会被执行，你不定义，系统就送一个隐式的构造函数，但啥也不干

成员函数会被最终编译成与全局无关的函数
但成员变量的作用域不是全局，无法在函数内部中访问这些成员变量
所以C++编译时，会额外添加一个参数，把当前对象的指针传递进去，通过指针来访问成员变量

## 对象的内存模型

In [ ]:
//通过对象指针来访问类的成员 
Student *pStu = new Student; //在堆上给Student类分配内存空间，并由对象指针指针pStu来保存
cout << sizeof(Student) << endl;
cout << sizeof(*pStu) << endl;
delete pStu;

In [ ]:
//通过对象来访问类的成员 
Student stu_1;  //在栈上创建对象
cout << sizeof(stu_1) << endl;

私有权限的成员变量无法在外部通过对象来调用,所以不改成员变量的访问权限就没办法直接对成员变量赋值

In [ ]:
pStu->m_name = "小明"; //通过对对象使用字符指针变量来引用类的成员变量
pStu->m_age = 15;	
pStu->m_score = 92.5f;
pStu->say();

然而有构造函数就可以赋初值了

In [8]:
//堆上分配的构造函数调用
Student *pStu2 = new Student("李华", 16, 96);
pStu2->say();

李华的年龄是16成绩是96


In [11]:
//栈上分配的对象构造函数的调用
Student stu_2("张红", 16, 94.5f);
stu_2.say();

张红的年龄是16成绩是94.5


默认构造函数

In [ ]:
Student *pStu3 = new Student();  //默认构造函数的调用
pStu3->show();

In [ ]:
Student stu_3();
//括号可以省略
Student stu_3;

使用初始化列表

In [ ]:
Student::Student(string name, int age, float score):m_name(name), m_age(age), m_score(score)
{
}

In [ ]:
Student::Student(string name, int age, float score):m_name(name)
{
    m_age = age;
    m_score = score;
}

In [ ]:
VLA::VLA(int len): m_len(len)
{
    if(len > 0)
    {
        m_arr = new int[len];
    }
    else
    {
        m_arr = NULL;
    }
}
VLA::~VLA
{
    delete[] m_arr;
}

# 拷贝构造函数

In [ ]:
#include <iostream>
using namespace std;

class Integer
{
private:
    int *m_pi;   //私有成员变量  指向整型数据的指针变量
public:
    Integer(int i = 0)  //公有构造函数 ，默认参数为0
    {
        m_pi = new int(i);   //动态分配内存,将i的地址赋给m_pi
    }
    void print(void)   //公有成员函数
    {
        cout << *m_pi << endl;  //输出m_pi指向的整型数据
    }
    ~Integer()  //析构函数
    {
        delete m_pi;  //释放内存
    }
};
int main(void)
{
    Integer i1(100);  //创建对象i1 调用构造函数，将100赋给m_pi
    Integer i2 = i1;  //创建对象i2 调用拷贝构造函数，将i1的地址赋给i2
    Integer i3(i1);   //创建对象i3 拷贝构造函数 ，将i1的地址赋给i3
    i2.print();
    i3.print();
    return 0;
}

这段代码中，在构造对象i2 和 i3 时 ，并没有按照常规的做法传入一个整型值
对i2 是直接把 i1 作为右值
对i3 是将i1作为参数传入
这样是可以的吗？

当两个对象进行赋值操作时，比如“i2=i1”编译器会将其处理为“i2.operator=(i1) 的成员函数调用形式”,其中“operator=”称为拷贝赋值操作符函
数，由该函数完成其赋值运算，其返回结果就是表达式的结果。
如果没有自己定义拷贝赋值操作符函数，编译器会为该类提供缺省的拷贝赋值操作符函数，用于完成两个对象的赋值操作。

# 静态成员

C++中希望某个`类的多个对象之间`实现数据共享，可以通过static建立一个`被局限在类中使用`的全局资源，该类型资源被称为`静态成员`。

## 静态成员变量（可以理解为局限在类中使用的全局变量）

> 特征

在类的定义中，用static修饰的成员变量，称为静态成员变量。

```
class 类名
{
    static 数据类型 变量名;//声明
};
数据类型 类名::变量名 = 初值;//定义和初始化
```

> 内存模型

实例化对象时只实现非静态成员变量

![alt text](<截图 2024-05-29 23-49-36.png>)

>访问方式：  

1. 可以直接通过类名来使用
   
   类名::静态成员变量; 

2. 可以创建完对象后，通过对象来使用
   
   对象.静态成员变量;  

In [ ]:
#include <iostream>
using namespace std;
class A
{
public:
    int m_data;
    static int s_data;  //声明静态成员变量,可以供所有由该类创建的对象访问
    A(int data=0):m_data(data){}
};
int A::s_data = 100; //定义静态成员变量并对其初始化

int main(void)
{
    cout << A::s_data << endl;  //通过类名直接访问静态成员变量
    A a1(123);  //创建对象a1
    cout << "a1 size =" << sizeof(a1) << endl; 
    cout << a1.s_data << endl;  //通过对象名访问静态成员变量
    A a2(1);  //创建对象a2
    a2.s_data = 999;  //通过对象名修改静态成员变量
    cout << a1.s_data << endl;  // 这里可以看到a1的静态成员变量值被a2修改了,说明静态成员变量是所有对象共享的
    return 0;
}

## 静态成员函数(为被限制在类中使用的全局函数)

> 特征

被static修饰的成员函数即为静态成员函数。

```
class 类名
{
访问控制限定符:
static 返回类型 函数名(形参表) { ... }
};
```



注意：
1. 静态成员函数可以直接定义在类的内部，也可以定义在类的外部，这一点和普通的成员函数没有区别。
   
2. 静态成员函数没有this指针(不用区分是哪个对象)，没有const属性，可以把静态函数理解为被限制在类中使用的全局函数.

3. 静态成员函数中只能访问静态成员(静态成员函数天生是为静态成员变量服务的)，但是在非静态成员函数中既可以访问静态成员也可以访问非静态成员*（静态成员变量被所有对象共享，成员函数也是供所有对象调用的(通过this指针区分不同对象)）

4. 静态成员函数和静态成员变量一样，也要受到类的访问控制限定符的约束

> 调用形式

在类的外部访问静态成员函数

1. 通过类名来使用 
   
   类名::静态成员函数(实参表);
2. 通过对象来使用
   
   对象.静态成员函数(实参表);

In [ ]:
#include <iostream>
using namespace std;

class A
{
public:
    int m_data;
    static int s_data;

    A(int data=0):m_data(data){}  //构造函数
    void func1(void)  // func1(A *this)
    { 
    cout << m_data << endl;
    cout << s_data << endl;
    }
    static void func2()  //静态成员函数
    {  
        cout << "静态成员函数" << endl;
        cout << s_data << endl;  //静态成员函数中可以访问静态成员变量
        //cout << m_data << endl; //error  //静态成员函数中不能使用非静态成员变量
    }
};
int A::s_data = 100; //定义 并 初始化
int main(void)
{
    cout << A::s_data << endl;  //通过类名访问静态成员变量
    A::func2();  //通过类名访问静态成员函数

    //A::func1(); //func1(&对象的地址) error  //对一般的成员函数，需要先创建对象，传入this指针
    return 0;
}

# 友元

类的封装具有信息隐藏能力，但也带来了访问效率的问题.  
c++通过友元给某些函数或者类一项`特权`，可以访问类中的私有成员，使用的关键字是`friend`
让一个类具有对外开放的特性，哪个类要开放在哪个类中，使用`friend`关键字

## 友元函数

这里主要是指那些在类外定义的、不属于当前类的一般意义上函数。让这些函数能访问到类内的私有成员。

In [ ]:
#include <iostream>
using namespace std;
class Student
{
public:
    Student(char *name, int age, float score);
public:
    friend void show(Student *pstu);  // 使用friend关键将show函数声明为友元函数,给show函数开了后门
private:
    char *m_name;
    int m_age;
    float m_score;
};
Student::Student(char *name, int age, float score):m_name(name), m_age(age), m_score(score){}

void show(Student *pstu)  // 一般意思上的函数，通过在类中使用friend声明，使得其可以访问类中的私有成员
{
    cout << pstu->m_name << "的年龄是" << pstu->m_age << "的分数是" << pstu->m_score << endl;
    //但是友元函数不同于类的成员函数，它没有this指针，要通过对象名来访问成员变量，要告诉编译器，用的是哪个对象的属性
    //能访问，但不能直接访问，本来是连访问都不允许的
}

int main()
{
    Student stu("张三", 18, 90.5);
    show(&stu);
    Student *pstu = new Student("李四", 19, 80.5);
    show(pstu);

    return 0;
}

## 友元成员函数


对一个类，可以指定它的某个`成员函数`是另一个类的友元，也就是`友元成员函数`。

In [ ]:
#include <iostream>
using namespace std;

class B
{
private:
    int z;
public:
    B(int i = 0)  //默认构造函数
    {
        z = i;
    }
    int add(const A&);
    int sub(const A&);
};
class A
{
private:
    int x, y;
public:
    A(int i, int j)
    {
        x = i;
        y = j;
    }
    int getx(void)
    {
        return x;
    }
    int gety(void)
    {
        return y;
    }
    //-------------------------//
    friend int B::add(const A& a);  //在A的类定义中 声明B中的add函数是A的友元函数，让B中的add函数可以访问A中的私有成员
    //-------------------------//
};
int B::add(const A& a)
{
    return a.x + a.y + z;
}
int B::sub(const A& a)
{
    //
    return a.x - a.y - z;
}
int main(void)
{
    A a(2,3);
    B b(4);
    cout << b.add(a) << endl;
    cout << b.sub(a) << endl;
    return 0;
}

## 友元类

一个类可以是另一个类的`友元类`，友元类的`所有成员函数`都是另一个类的友元函数，能够直接访问另一个类的所有成员。

In [ ]:
#include <iostream>
using namespace std;

class A
{
private:
    int x, y;
public:
    A(int i, int j)
    {
        x = i;
        y = j;
    }
    int getx(void)
    {
        return x;
    }
    int gety(void)
    {
        return y;
    }
    //--------------------------//
    friend class B; //声明B是A的友元类,让B的所有成员函数可以访问A的私有成员
    //--------------------------//
};

class B
{
private:
    int z;
public:
    B(int i = 0)
    {
        z = i;
    }
    int add(const A& a)  //const修饰的参数a是常量引用
    {
        return a.x + a.y + z;  //a.x, a.y是A的私有成员， B的成员函数可以访问
    }
    int sub(const A& a)  //const修饰的参数a是常量引用
    {
        return a.x - a.y - z;  //a.x, a.y是A的私有成员  B的成员函数可以访问
    }
};

int main(void)
{
    A a(2,3);
    B b(4);
    cout << b.add(a) << endl;
    cout << b.sub(a) << endl;
    return 0;
}

注意这种友元关系并不是双向的，哪个类要对外开放，在哪个类中声明即可。

# 单例模式

单例模式(Singleton Pattern，也称为单件模式)，使用最广泛的设计模式之一。其意图是保证一个类仅有一个实例，并提供一个访问它的全局
访问点，该实例被所有程序模块共享。

面向对象编程中，每个对象都应该抽象代表一个设备，并通过对象完成对某个具体设备的管理和维护。

对于有些类只能有一个实例很重要，例如打印机管理器、设备管理器、任务管理器等。

## 实现单例的三个步骤

私有化构造函数   
其意义在于防止其他类实例化该类

In [ ]:
class Singleton
{
private:  
    //这里的构造函数对外部是隐藏的,类外不能直接创建对象
    Singleton(void){...}  // 构造函数
    Singleton(const Signleton &that){}  // 拷贝构造函数,传入参数是Signleton对象的const引用
...
};

使用静态成员变量维护唯一的单例对象


In [ ]:
class Singleton
{
private:
    Singleton(void){...}
    Singleton(const Signleton &that){}
...
    static Singleton s_instance;  // 静态成员变量,这个变量是类的一个对象，也就是这个类的一个实例
};
Singleton Singleton::s_instance;  // 静态成员变量初始化



定义静态成员函数用于获取单例对象


In [ ]:
class Singleton
{
private:
    Singleton(void){...}
    Singleton(const Signleton &that){}
...
    static Singleton s_instance;
public:
    static Singleton& getInstance(void)  // 只能通过getInstance()来访问 ，这里getInstance()的类型是Singleton&
    {
        return s_instance;
    }
};
Singleton Singleton::s_instance;

## 饿汉式

加载进程时即完成创建（饿）, 用不用都创建。

In [ ]:
#include <iostream>
using namespace std;

class Signleton
{
private:
    int m_i;
    Signleton(int i=12345)
    {
        m_i = i;
    }
    Signleton(const Signleton& that){}  //1. 私有化构造函数
    static Signleton m_instance;        //2. 私有化静态成员, 用于存储实例
public:
    static Signleton& getInstance()     //3. 声明静态成员函数，用于获取实例
    {
        return m_instance;
    }
    void print()
    {
        cout << m_i << endl;
    }
};
Signleton Signleton::m_instance = 111;

int main(void)
{
    Signleton& s1 = Signleton::getInstance();  //创建了实例s1，是m_instance的引用
    Signleton& s2 = Signleton::getInstance();  //创建了实例s2，是m_instance的引用
    Signleton& s3 = Signleton::getInstance();  //创建了实例s3，是m_instance的引用
    //这里s1,s2,s3是同一个实例
    //Signleton s4 = 12; //error  //不能使用构造函数创建实例
    cout << &s1 << endl;
    cout << &s2 << endl;
    cout << &s3 << endl;
    s1.print();
    s2.print();
    s3.print();
    return 0;
}
/*
++++++++++++++++++++++++++++++++++++
返回结果
0x5675f77d8154
0x5675f77d8154
0x5675f77d8154
12345
12345
12345
可见，s1,s2,s3是同一个实例。
++++++++++++++++++++++++++++++++++++
*/

## 懒汉式

用时再创建（懒），不用再销毁

In [ ]:
#include <iostream>
using namespace std;

class Signleton
{
private:
    int m_i;
    static int m_count ; //记录对象的引用次数
    static Signleton *m_instance;  //只一个对象的指针
    Signleton(int i = 0)
    {
        m_i = i;
        cout << "constructor " << endl;
    }
    Signleton(const Signleton& that){}
public:
    static Signleton& getInstance(void)
    {
        if(m_instance == NULL)
        {
            m_instance = new Signleton(123);
        }
        m_count++;
        return *m_instance;
    }
    void release()  //释放对象
    {
        m_count--;
        if(m_count == 0)  //如果引用次数为0，则释放对象
        {
            delete m_instance;
            m_instance = NULL;
        }
    }
    ~Signleton()  //析构函数
    {
        cout << "destroy" << endl;
    }
};
Signleton* Signleton::m_instance = NULL;  //初始化
int Signleton::m_count = 0;

int main(void)
{
    //Signleton s1; //error  
    Signleton& s1 = Signleton::getInstance();
    Signleton& s2 = Signleton::getInstance();
    Signleton& s3 = Signleton::getInstance();
    cout << &s1 << " " << &s2 << " " << &s3 << endl;
    s1.release();
    s2.release();
    s3.release();
    return 0;
}
/*
++++++++++++++++++++++++++++
constructor 
0x624aa08e8eb0 0x624aa08e8eb0 0x624aa08e8eb0
destroy
*/

# 继承

## 什么是继承，以及有什么用？

继承，基于一个已有类创建新类，使新类与已有类具有同样的功能，即新类具有已有类相同的数据成员和成员函数

继承是实现代码重用的重要方法

通过继承可以避免代码的重复定义  

已有类称为基类（父类/超类），新类称为派生类（子类）

## 哪些可以继承，哪些不能继承？

派生类继承基类后，派生类将自动获得基类的所有成员，但是继承也有一定的限制：

注意：

基类的构造函数和析构函数不能继承

基类的友元函数不能继承

静态数据成员和静态成员函数不能继承

## 继承的方式

> 公有继承

In [ ]:
#include <iostream>
using namespace std;

class Base
{
private:
    int m_private;
protected:
    int m_protected;
public:
    int m_public;
    
    Base(int a=1, int b=2, int c=3)
    {
        m_private = a;
        m_protected = b;
        m_public = c;
    }
    int geta()  
    {
        return m_private;
    }
};
class Derived: public Base  //以公有方式继承基类
{
public:
    void print()  //一般的成员函数
    {
        //cout << m_private << endl; //error //不能访问基类的私有成员
        cout << geta() << endl;  //可以通过基类的公有成员函数访问基类的私有成员
        cout << m_protected << endl;  //可以访问基类的保护成员
        cout << m_public << endl;  //可以访问基类的公有成员
        //public 继承比较平凡，对基类的成员的权限没有影响
    }
};

int main(void)
{
    Derived test;  //创建子类对象
    //cout << test.m_private << endl; //error private
    //cout << test.m_protected << endl;// error protected  
    //protected 权限保护类内成员不能由类外访问，但子类可以访问
    cout << test.m_c << endl;
    return 0;
}

> 保护继承

In [ ]:
#include <iostream>
using namespace std;

class Base
{
private:
    int m_private;
protected:
    int m_protected;
public:
    int m_public;
    
    Base(int a=1, int b=2, int c=3)
    {
        m_private = a;
        m_protected = b;
        m_public = c;
    }
    int geta()
    {
        return m_private;
    }
};
class Derived: protected Base
{
public:
    void print()
    {
        //cout << m_private << endl; //error  //保护继承不能访问基类中的私有变量
        cout << geta() << endl;
        cout << m_protected << endl;
        cout << m_public << endl;
    }
};
int main(void)
{
    Derived test;
    //cout << test.m_private << endl; // error private
    //cout << test.m_protected << endl; // error protected
    cout << test.m_public << endl; //protected继承 子类中该变量为protected
    return 0;
}

> 私有继承

In [ ]:
#include <iostream>
using namespace std;

class Base
{
private:
    int m_a;
protected:
    int m_b;
public:
    int m_c;
    
    Base(int a=1, int b=2, int c=3)
    {
        m_a = a;
        m_b = b;
        m_c = c;
    }
    int geta()
    {
        return m_a;
    }
};

class Derived: private Base
{
public:
    void print()
    {
        //cout << m_a << endl; //error
        cout << geta() << endl;
        cout << m_b << endl;
        cout << m_c << endl;
    }
};

int main(void)
{
    Derived test;
    //cout << test.m_a << endl; //error private
    //cout << test.m_b << endl; // private继承 error private
    //cout << test.m_c << endl; //private继承 子类中该变量为private
    return 0;
}

### 继承中的访问权限问题

![alt text](image.png)

可见  
公有继承不改变继承来的基类成员的权限  
保护继承则把低于保护的权限升级成保护  
私有继承则把低于私有的权限升级成私有  

总结一下权限的相关问题：

权限的引入体现了C++语言面向对象编程的其中一个重要特性：封装， 会通过关键字来限制类的外部对成员的访问， 从而保证类的内部实现不受外界干扰。

public权限： 并不保护成员， 任何地方都可以访问
protected权限： 保护成员， 只有本类和子类的成员函数可以访问，可以认为是private给子类特别开放的权限
private权限： 最高禁令， 只有本类可以访问

![alt text](image-1.png)

## 基类与派生类的关系

子类对象会继承基类的属性的行为，通过子类对象可以直接访问基类中的成员，如同是基类对象在访问它们一样

![alt text](image-2.png)

向上造型(upcast)：将`子类类型`的指针或引用转换为`基类类型`的指针或引用；这种`操作性缩小`的类型转换，在编译器看来是安全的，可以隐式转换

子类 ————> 基类

向下造型(downcast)：将`基类类型`的指针或引用转换为`子类类型`的指针或引用；这种`操作性放大`的类型转换，在编译器看来是危险的，不能隐式转化，但是可以显式转换

基类 ————> 子类

In [ ]:
#include <iostream>
using namespace std;

class Human
{
private:
    int m_private;
protected:
    string m_name;
    int m_age;
    const int& get(void)  //返回函数的类型是const int&是常引用，不能修改
    {
        return m_private;
    }
public:
    Human(const string &name, int age)
    {
        m_name = name;
        m_age = age;
        m_private = 1234;
    }
    void eat(const string& food)
    {
        cout << "我在吃: " << food << endl;
    }
    void sleep(int hour)
    {
        cout << "我睡了" << hour << "小时" <<endl;
    }
};
class Student: public Human  //从Human公有继承来的子类Student
{
private:
    int m_no; //学号
public:
    Student(const string& name, int age, int no):Human(name, age)  //调用父类构造函数
    // 子类也有name和age两个成员变量，但是父类中已经有了，所以不需要再定义
    // 子类的构造函数中，必须调用父类构造函数，对父类成员进行初始化
    {
        m_no = no;
    }
    void who(void)
    {
        cout << "我叫: " << m_name << ", 今年" <<m_age<<"岁，学号是: "<<m_no << endl;
        //cout << m_private << endl; //error
        // 公有继承下，子类可以直接访问父类的成员，但是私有成员不能访问
        cout << get() << endl;  //公有继承可以访问父类的私有成员
    }
    void learn(const string& course)  //传入的参数是const string&是常引用
    {
        cout << "我在学" << course << endl;
    }
};
class Teacher: public Human
{
private:
    int m_salary;
public:
    Teacher(const string& name, int age, int salary):Human(name, age),m_salary(salary){}
    void teach(const string& course)
    {
        cout << "我正在讲 " << course << endl;
    }
    void who(void)
    {
        cout << "我叫 "<<m_name << ",今年" << m_age << "岁, 工资是" << m_salary << endl;
    }
};
int main(void)
{
    Student s("张飞", 28, 100011);
    cout << "sizeof(s) = "<<sizeof(s) << endl;
    s.who();  // 学生子类使用基类的成员函数
    s.eat("宫保鸡丁");  //学生子类使用基类的成员函数
    s.sleep(8);  //学生子类使用基类的成员函数
    s.learn("C++编程");  //学生子类使用自己的成员函数

    Teacher t("诸葛亮", 34, 200000);
    t.who();  //老师子类使用基类的成员函数
    t.teach("嵌入式");  //老师子类使用自己的成员函数
    t.sleep(7);  //老师子类使用基类的成员函数
    t.eat("汉堡");  //老师子类使用基类的成员函数
    
    // Student * -----> Human *:向上造型
    Human *ph = &s; //将Student对象地址赋值给Human指针，这属于向上造型
    ph->eat("香蕉");  
    ph->sleep(10);
    //ph->who();  // 这里向上造型后，无法使用子类自己的成员函数
    //error

    // Human * --------> Student *: 向下造型（合理）
    Student *ps = static_cast<Student *>(ph);  
    ps->who();

    Human h("赵云", 22);
    //Human * -------> Student *: 向下造型 (不合理)
    Student *ps2 = static_cast<Student *>(&h);
    ps2->who();
    return 0;
}

### 基类的派生类的名字隐藏问题 

子类中定义了和父类的同名函数，对父类的成员函数造成了隐藏

In [ ]:
#include <iostream>
using namespace std;

class Base
{
private:
    int x;
public:
    void set(int i)
    {
        x = i;
    }
    void print()
    {
        cout << "Base class " << "x= " << x << endl;
    }
};
class Derived: public Base
{
private:
    int m, n;
public:
    void set(int p, int k)
    {
        m = p;
        n = k;
    }
    void print()   //派生类中也有print函数
    {
        Base::print();  //调用基类中的同名函数
        cout << "Derived class "<< "m = "<< m <<", n=" << n << endl;
    }
};
int main(void)
{
    Derived d;
    d.set(10,20);
    //d.set(100); // error名字隐藏  派生类的对象d中的set函数隐藏了基类的同名函数，不能传入两个参数
    d.Base::set(100);  //调用基类中的set函数需要加作用域限定符
    d.print();  //调用派生类中的print函数
    return 0;
}

## 派生类的对象创建

### 构造函数



因为在构造派生类的过程中，必须先构造基类对象，而基类对象又依赖于派生类对象的成员变量
所以，构造派生类对象时，先构造基类对象，再构造成员对象

如果子类构造函数没有显式指明基类部分的初始化方式，那么编译器将会自动调用基类的无参构造函数来初始化基类子对象

如果希望以有参的方式来初始化基类部分，那么必须使用初始化列表来显式指明子对象构造顺序

+ 分配内存
+ 构造基类子对象(按继承表顺序)
+ 构造成员子对象(按声明顺序)
+ 执行子类构造函数代码


In [ ]:
#include <iostream>
using namespace std;

class Member
{
private:
    int m_k;
public:
    Member()
    {
        cout << "Member()" << endl;
        m_k = 0;
    }
    Member(int k)
    {
        cout << "Member(int)" << endl;
        m_k = k;
    }
};
class Base
{
private:
    int m_i;
public:
    Base()
    {
        cout << "Base()" << endl;
        m_i = 0;
    }
    Base(int i)
    {
        cout << "Base(int)" << endl;
        m_i = i;
    }
};
class Derived:public Base
{
private:
    int m_j;
    Member m_m;  // 派生类中包含一个其他类的对象
public:
    Derived():Base(100),m_m(200)
    {
        cout << "Derived()" << endl;
    }
    Derived(int i, int j):Base(i)   //注意这里可以使用形参列表中的i
    {
        cout << "Derived(int, int)" << endl;
        m_j = j;
    }
};
int main(void)
{
    Derived d1;  
    // 创建派生类的对象，调用默认构造函数，
    // 这时会使用派生类的无参构造函数中的初始化列表调用基类有参构造函数，
    // 对Member对象也调用其有参构造函数进行初始化
    Derived d2(1,2);
    // 创建派生类的对象，调用有参构造函数，
    // 这时会使用派生类的有参构造函数中的初始化列表调用基类有参构造函数，
    // 对Member对象没有对其指定初始化的值，调用其无参构造函数进行初始化
    return 0;
}
/* 输出结果：
Base(int)
Member(int)
Derived()
可以看到是先调用了基类的构造函数，再调用派生类构造函数
Base(int)
Member()
Derived(int, int)
*/

### 析构函数

子类的析构函数，无论自己定义的，还是编译器缺省提供，都会自动调用基类的析构函数，完成基类子对象的销毁。

子类对象销毁过程:

+ 执行子类析构函数代码
+ 析构成员子对象(按声明逆序)
+ 析构基类子对象(按继承表逆序)
+ 释放内存


In [ ]:
#include <iostream>
using namespace std;

class Member
{
private:
    int m_k;
public:
    Member()
    {
        cout << "Member()" << endl;
        m_k = 0;
    }
    Member(int k)
    {
        cout << "Member(int)" << endl;
        m_k = k;
    }
    ~Member()
    {
        cout << "~Member()" << endl;
    }
};
class Base
{
private:
    int m_i;
public:
    Base()
    {
        cout << "Base()" << endl;
        m_i = 0;
    }
    Base(int i)
    {
        cout << "Base(int)" << endl;
        m_i = i;
    }
    ~Base()
    {
        cout << "~Base()" << endl;
    }
};
class Derived:public Base
{
private:
    int m_j;
    Member m_m;
public:
    Derived():Base(100),m_m(200)
    {
        cout << "Derived()" << endl;
    }
    Derived(int i, int j):Base(i)
    {
        cout << "Derived(int, int)" << endl;
        m_j = j;
    }
    ~Derived()
    {
        cout << "~Derived()" << endl;
    }
};
int main(void)
{
    Derived d1;
    Derived d2(1,2);
    return 0;
}
/*
//d1
Base(int)
Member(int)
Derived()
//d2
Base(int)
Member()
Derived(int, int)
//~d1
~Derived()
~Member()
~Base()
//~d2
~Derived()
~Member()
~Base()
*/

## 多重继承

C++允许一个类从一个或多个基类派生。如果一个类只有一个基类，称为单一继承。如果一个类具有两个或两个以上的基类，就称为多重继承

### 多重继承的使用方法

In [ ]:
#include <iostream>
using namespace std;
class Phone
{
private:
    string m_number;
public:
    Phone(const string& number)
    {
        m_number = number;
    }
    void call(const string& number)
    {
        cout << m_number << "打给： " << number << endl;
    }
};
class Player
{
public:
    Player(const string& media)
    {
        m_media = media;
    }
    void play(const string& music)
    {
        cout << m_media << "正在播放: " << music << endl;
    }
private:
    string m_media ; //播放器的名称
};
class Computer
{
private:
    string m_os; //使用的操作系统
public:
    Computer(const string& os):m_os(os){}
    void run(const string& app)
    {
        cout << "在" << m_os << "正在运行: " << app << endl;
    }
};
/*典型的多重继承*/
class SmartPhone:public Phone, public Player, public Computer
{
public:
    SmartPhone(const string& number, const string& media, const string& os):Phone(number), Player(media),Computer(os){}
};

int main(void)
{
    SmartPhone huawei("13988888888", "MP4", "鸿蒙");
    huawei.call("010-12345");
    huawei.play("我和我的祖国");
    huawei.run("王者荣耀");
    return 0;
}


### 命名冲突


当两个不同基类拥有同名成员时，容易产生名字冲突问题。
使用域限定符解决。

In [ ]:
#include <iostream>
using namespace std;
class A
{
public:
    void func(void)
    {
        cout << "A::func()" << endl;
    }
};
class B
{
public:
    void func(void)
    {
        cout << "B::func()" << endl;
    }
};
class C: public A, public B
{
};
int main(void)
{
    C test;
    test.A::func();  //指定使用基类A的func
    return 0;
}

### 钻石继承和虚继承

钻石继承，一个派生类继承的多个基类又源自一个公共的祖先（公共基类）


In [ ]:
#include <iostream>
using namespace std;

class A
{
protected:
    int m_data;
public:
    A(int data)
    {
        m_data = data;
        cout << "A(int)" << endl;
    }
};
class B: public A
{
public:
    B(int data):A(data)
    {
        cout << "B(int)" << endl;
    }
    void set(int data)
    {
        m_data = data;
    }
};
class C: public A
{
public:
    C(int data):A(data)
    {
        cout << "C(int)" << endl;
    }
    int get(void)
    {
        return m_data;
    }
};
class D: public B, public C
{
public:
    D(int data): B(data), C(data)
    {
        cout << "D(int)" << endl;
    }
};

int main(void)
{
    D d(100);
    cout << sizeof(d) << endl;
    cout << d.get() << endl; // 100  //调用C的get方法，获取m_data的值
    d.set(200);  //调用B的set方法，修改m_data的值
    cout << d.get() << endl; //100  //调用C的get方法，获取m_data的值
    return 0;
}
/*
//创建D类对象d，对A的构造函数进行了两次调用
A(int)
B(int)  // 初始化B要调用一次A的构造函数
A(int)
C(int)  // 初始化C要调用一次A的构造函数
D(int)

8  // A类成员变量是4个字节，B类成员变量是4个字节，C类成员变量是4个字节
100
100

虽然 set 方法被调用了，但由于多重继承的复杂性，C 中的 m_data 可能并没有被修改，
因此输出的值可能仍然是 100，而不是预期的 200。
这是因为 B 和 C 各自继承了 A 的一个独立副本，set 方法修改的是 B 继承的那部分的 m_data，
而 get 方法访问的是 C 继承的那部分的 m_data

由于 D 同时从 B 和 C 继承，而 B 和 C 又都从 A 继承，导致 D 对象中实际上包含了两个 A 的子对象。
这可能导致一些意料之外的行为，特别是当试图通过不同的路径（如 B 和 C）访问或修改相同的成员变量时。
*/

解决这种对共有的基类继承有两种不同路径的方式式：虚继承

虚继承语法：

在继承表使用virtual关键字修饰

位于继承链末端子类负责构造公共基类子对象

In [ ]:
#include <iostream>
using namespace std;

class A
{
protected:
    int m_data;
public:
    A(int data)
    {
        m_data = data;
        cout << "A(int)" << endl;
    }
};
class B: virtual public A //添加virtual关键字，指定中间继承类为虚基类
{
public:
    B(int data):A(data)
    {
        cout << "B(int)" << endl;
    }
    void set(int data)
    {
        m_data = data;
    }
};
class C: virtual public A
{
public:
    C(int data):A(data)
    {
        cout << "C(int)" << endl;
    }
    int get(void)
    {
        return m_data;
    }
};
class D: public B, public C
{
public:
    D(int data): B(data), C(data), A(data)
    {
        cout << "D(int)" << endl;
    }
};
int main(void)
{
    D d(100);
    cout << sizeof(d) << endl;
    cout << d.get() << endl; // 100
    d.set(200);
    cout << d.get() << endl; //200
    return 0;
}

## 继承与组合

继承与组合是C++实现代码重用的两种主要方法。

继承是Is-a的关系，比如水果和梨

组合是Has-a的关系，图书馆有图书

In [ ]:
#include <iostream>
using namespace std;

class vehicles
{ //交通工具
public:
    void load(const string& goods)
    {
        cout << "装载" << goods << endl;
    }
};
class tyre
{ //轮胎
public:
    void run(const string& dest)
    {
        cout << "转动方向: " << dest << endl;
    }
};
class car: public vehicles  //car类和vehicles是继承关系
{
public:
    tyre wheel;  // car类和tyre类是组合关系
};

int main(void)
{
    car c1;
    c1.load("电脑");
    c1.wheel.run("北京");
    return 0;
}

## 多文件编程实例

一般将类的声明放在.h文件中，类中成员函数的定义放在.cpp文件中。

In [ ]:
/*person.h*/
#ifndef __PERSON_H__
#define __PERSON_H__
#include <iostream>
using namespace std;
class person
{
private:
    int age;
    string name;
public:
    person(int age, const string& name);
    void whoami(void);
};
#endif


In [ ]:
//函数的定义单独用一个文件
/*person.cpp*/
#include "person.h"
person::person(int age, const string& name)
{
    this->age = age;
    this->name = name;
}
void person::whoami(void)
{
    cout << "我是: " << name << endl;
}


In [ ]:
//不同的类放到不同的文件中
/*student.h*/
#ifndef __STUDENT_H__
#define __STUDENT_H__
#include "person.h"
class student: public person
{
private:
    float score;
public:
    student(int age, const string& name, float score);
    void whoami(void);
};
#endif


In [ ]:

/*student.cpp*/
#include "student.h"
student::student(int age, const string& name, float score):person(age,name)
{
    this->score = score;
}
void student::whoami(void)
{
    person::whoami();
    cout << "我的成绩是: " << score << endl;
}


In [ ]:

/*main.cpp*/
#include "student.h"
int main(void)
{
    student s1(22, "刘备", 81.5);
    s1.whoami();
    return 0;
}


In [ ]:
g++ person.cpp student.cpp main.cpp
./a.out

# 多态

## 多态是什么？

多态（polymorphism）通俗地讲，就是用一个相同的名字定义许多不同的函数，这些函数
可以针对不同数据类型实现相同或相似的功能，即所谓的“一个接口，多种实现”。

In [ ]:
#include <iostream>
using namespace std;
class Shape   //形状类
{
public:
    virtual void draw(void)  //有方法draw
    {
        cout << "draw shape" << endl;
    }
};
class Rect:public Shape  //三角类
{
public:
    void draw(void)  //有方法draw
    {
        cout << "draw Rect" << endl;
    }
};
class Circle: public Shape  //圆形类
{
public:
    void draw(void)  //有方法draw
    {
        cout << "draw Circle" << endl;
    }
};
class Ellipse: public Shape  //椭圆类
{
public:
    void draw(void)  //有方法draw
    {
        cout << "draw Ellipse" << endl;
    }
};
int main(void)
{
    /*
    Ellipse e;  //创建对象e  
    e.draw();  //调用draw方法
    e.Shape::draw();  //调用父类的方法
    */
    Shape *buf[128] = {0};  //创建一个指向Shape类的指针数组变量buf，一共有128个元素，每个元素初始化为0
    buf[0] = new Rect;  //创建对象Rect，并把对象的地址赋给buf[0]
    buf[1] = new Circle;  //创建对象Circle，并把对象的地址赋给buf[1]
    buf[2] = new Ellipse;   //创建对象Ellipse，并把对象的地址赋给buf[2]
    for(int i=0; buf[i] != NULL; i++)
    {
        buf[i]->draw();  //draw方法将表现出不同的行为
    }
    return 0;
}
/*
draw Rect
draw Circle
draw Ellipse
一个指向基类的指针可以指向一个派生类对象，使得重载的函数具有不同的行为
*/

## 虚函数

被virtual关键字修饰的`成员函数`称为`虚函数` (注意要和修饰类区别开来)

基类中声明虚函数，子类中也有一个同名的函数

> 函数重写

如果将`基类`中的某个成员函数声明为虚函数，那么`子类`中与该函数具有相同原型的成员函数就也是虚函数，并且对基类中版本形成覆盖，即`函数重写`

> 虚函数覆盖

如果子类提供了对基类虚函数有效的覆盖，那么通过`指向子类对象的基类指针`，或者通过`引用子类对象基类引用`，调用该虚函数，实际被执行将是子类中的覆盖版本，而不再是基类中原始版本，这种语法现象被称为`多态`.

多态的意义在于，一般情况下，调用哪个类的成员函数由调用者指针或者引用本身类型决定的， 而有了多态，调用哪个类的成员函数由`调用者指针或者引用实际目标对象的类型`决定。

这样一来，源自同一种类型的同一种激励，竟然可以产生多种不同的响应，也就是对于同一个函数调用，能够表达出不同的形态，即为多态。

> 虚函数覆盖的条件：

1. 只有类中的成员函数才能声明为虚函数，而全局函数、静态成员函数、构造函数都不能被声明为虚函数

2. 只有在`基类中`以virtual关键字声明的虚函数，才能作为虚函数被子类覆盖

3. 而与`子类中`的virtual关键字无关虚函数在子类中的版本和基类中版本要具有相同的`函数签名`，即函数名、参数表、常属性一致

4. 如果基类虚函数返回基本类型的数据，那么子类中的版本必须返回相同类型的数据；如果基类虚函数返回类类型指针(A)或引用(A&)，那么允许子类中的版本返回其子类类型指针(B)或引用(B&)

In [ ]:
#include <iostream>
using namespace std;

class A{}; //类A
class B:public A{};  //类B继承自类A

class Base
{
public:
    virtual void func(void)   //基类中声明的虚函数
    {
        cout << "Base func" << endl;
    }
    virtual A* foo(void)  //基类中声明的虚函数，返回值是类指针
    {
        cout << "Base foo" << endl;
    }
};
class Derived: public Base
{
    void func(void)   //派生类中重载基类的虚函数
    {
        cout <<"Derived func" << endl;
    }
    B* foo(void)  //派生类中重载基类的虚函数
    {
        cout << "Derived foo" << endl;
    }
};

int main(void)
{
    Derived d1;  //派生类对象
    
    Base *pd1 = &d1;  //基类指针指向派生类的对象
    pd1->func();  //调用派生类中的虚函数

    Base &pd2 = d1;  //基类引用指向派生类的对象
    pd2.foo();  //调用派生类中的虚函数

    return 0;
}
/*
Derived func
Derived foo
*/

## 产生多态的条件：

1. 除了要满足函数重写的语法要求，还必须是通过指针或引用调用虚函数，才能表现出来

In [ ]:
#include <iostream>
using namespace std;

class A{};
class B:public A{};

class Base
{
public:
    virtual void func(void)
    {
        cout << "Base func" << endl;
    }
    virtual A *foo(void)
    {
        cout << "Base foo" << endl;
    }
};
class Derived: public Base
{
    void func(void) 
    {
        cout <<"Derived func" << endl;
    }
    B* foo(void)
    {
        cout << "Derived foo" << endl;
    }
};
int main(void)
{
    Derived d1;
    Base b = d1;
    b.func(); //调用 base中的func
    return 0;
}

2. 调用虚函数的指针也可以是this指针
   
   当使用子类对象调用基类中的成员函数时，
   该函数里面this指针将是一个`指向子类对象`的`基类指针`，再通过this去调用满足重写要求的虚函数同样可以表现多态的语法特性

In [ ]:
#include <iostream>
using namespace std;
class A{};
class B:public A{};

class Base
{
public:
    virtual void func(void)
    {
        cout << "Base func" << endl;
    }
    A* foo(void)
    {
        this->func();  //Derived func this是指向子类对象的基类指针
        //func();  //Derived func
    }
};
class Derived: public Base
{
    void func(void) 
    {
        cout <<"Derived func" << endl;
    }
};
int main(void)
{
    Derived d1;  //派生类对象
    d1.foo();  //调用基类的foo
    return 0;
}
/*
Derived func
*/

## 纯虚函数

如果一个虚函数仅表达抽象的行为，没有具体的功能，即只有声明没有定义，这样的虚函数被称为纯虚函数或抽象方法

这么做的目的在于，在基类中声明虚函数，在派生类中重写该虚函数，这样就可以实现多态。

```
class 类名 
{
public:
    virtual 返回类型 函数名 (形参表) = 0;
};
```

假设有图形类Figure, 设计计算面积的成员函数area()。Figure只是一个纯抽象意义上得概念，不存在计算面积或体积的具体方法，所以只能将成员函数area()设计为纯虚函数。

In [ ]:
#include <iostream>
using namespace std;

class Figure
{
protected:
    double x, y;
public:
    void set(double i, double j)
    {
        x = i;
        y = j;
    }
    virtual void area()=0;  //纯虚函数
};

## 抽象类

如果类中包含了纯虚函数，那么这个类就是抽象类,抽象类只能最为其它类的基类，不能用来建立对象。

如果类中的所有成员函数都是纯虚函数则可以称为纯抽象类

In [ ]:
#include <iostream>
using namespace std;

class Shape
{
public:
    virtual void draw(void) = 0;
};
class Rect:public Shape
{
public:
    void draw(void)
    {
        cout << "draw Rect" << endl;
    }
};
class Circle: public Shape
{
public:
    void draw(void)
    {
        cout << "draw Circle" << endl;
    }
};
class Ellipse: public Shape
{
public:
    void draw(void)
    {
        cout << "draw Ellipse" << endl;
    }
};
int main(void)
{
    /*
    Ellipse e;
    e.draw();
    e.Shape::draw();
    */
    //Shape s1; //error  抽象类不能实例化
    Shape *buf[128] = {0};
    buf[0] = new Rect;
    buf[1] = new Circle;
    buf[2] = new Ellipse;
    for(int i=0; buf[i] != NULL; i++)
    {
        buf[i]->draw();
    }
    return 0;
}

## 虚析构函数

C++允许将析构函数定义为虚函数，为什么？

In [ ]:
#include <iostream>
using namespace std;
class Base
{
public:
    Base()
    {
        cout << "Base 中通过 new 申请100个字节内存空间" << endl;
    }
    ~Base()
    {
        cout << "~Base 中通过 delete释放100个字节内存空间" << endl;
    }
};
class Derived:public Base
{
public:
    Derived()
    {
        cout << "Derived 中通过new 申请200个字节内存空间" << endl;
    }
    ~Derived()
    {
        cout << "~Derived 中通过delete 释放200个字节内存空间" << endl;
    }
};
int main(void)
{
    Base *pb = new Derived;
    delete pb; //只调用了基类的析构函数造成内存泄露
    return 0;
}
/*
Base 中通过 new 申请100个字节内存空间
Derived 中通过new 申请200个字节内存空间
~Base 中通过 delete释放100个字节内存空间
*/

如何解决该问题？将基类析构函数定义为虚函数

In [ ]:
#include <iostream>
using namespace std;

class Base
{
public:
    Base()
    {
        cout << "Base 中通过 new 申请100个字节内存空间" << endl;
    }
    virtual ~Base()  //基类的虚析构函数
    {
        cout << "~Base 中通过 delete释放100个字节内存空间" << endl;
    }   
};
class Derived:public Base
{
public:
    Derived()
    {
        cout << "Derived 中通过new 申请200个字节内存空间" << endl;
    }
    ~Derived()
    {
        cout << "~Derived 中通过delete 释放200个字节内存空间" << endl;
    }
};
int main(void)
{
    Base *pb = new Derived;
    delete pb; 
    return 0;
}
/*
Base 中通过 new 申请100个字节内存空间
Derived 中通过new 申请200个字节内存空间
~Derived 中通过delete 释放200个字节内存空间
~Base 中通过 delete释放100个字节内存空间
*/

## 虚函数的实现技术

In [ ]:
#include <iostream>
using namespace std;
class A
{
public:
    int m;
};
class B
{
public:
    int m;
    virtual void foo(void){}
};
int main(void)
{
    A a;
    B b;
    cout << "a size : " << sizeof(a) << endl;
    cout << "b size : " << sizeof(b) << endl;
    return 0;
}

每一个含有虚函数（无论是其本身的，还是继承而来的）的类 都至少有一个与之对应的`虚函数表`，

其中存放着该类所有的虚函数对应的函数指针。

![alt text](image-3.png)

当编译器编译以下test函数时只知道pb是B*类型的指针，并不知道它指向的具体对象类型 ：pb可能指向的是B的对象，也可能指向的是D的对象 

只有当程序执行过程中给test函数传递了具体参数才能确定pb指向了哪个对象，从而确定访问哪个虚表，从而实现了多态。

##  运行的类型信息

运行时类型信息（Run-time Type Information,RTTI）提供了`在程序运行时刻确定对象类型`的方法，是面向对象程序语言为解决多态问题而引入的一种语言特性。

由于多态的要求，C++指针或引用可能与他们实际代表的类型不一致（如基类指针可以指向派生类对象），当将一个多态
指针转换为其实际指向类型对象时，就需要知道对象类型信息。

在C++中，用于支持RTTI的运算符有：dynamic_cast, typeid, type_info.

### typeid和type_info


typeid操作符既可用于类型也可用于`对象`，返回typeinfo对象的常引用，用于表示`类型信息`

typeinfo类的成员函数name()，可以获取字符串形式的类型信息

In [ ]:
#include <iostream>
#include <typeinfo>
using namespace std;
int main(void)
{
    int i = 123;
    cout << typeid(i).name() << endl;
    cout << typeid(int).name() << endl;

    int * a1[10];
    int (*a2)[10];
    cout << typeid(a1).name() << endl;
    cout << typeid(a2).name() << endl;

    return 0;
}
/*
i
i
A10_Pi
PA10_i
*/

typeinfo类支持“==”和“!=”操作符，可直接用于类型相同与否的判断，如果类型之间存在多态的继承关系，typeid还可以利用多态的特性确定实际对象的类型

In [ ]:
#include <iostream>
#include <typeinfo>
using namespace std;

class A
{
public:
    virtual void foo(void){}
};
class B: public A
{
    void foo(void){}
};
class C: public A
{
    void foo(void){}
};
void func(const A& a)
{
    if(typeid(a) == typeid(B))
    {
        cout << "B" << endl;
    }
    else if(typeid(a) == typeid(C))
    {
        cout << "C" << endl;
    }
}
int main(void)
{
    B b;
    C c;
    func(b);
    func(c);
    return 0;
}

### dynamic_cast



强制类型转换运算符，主要用于具有多态继承关系父子类指针或引用之间的显式转换。
语法格式：

dynamic_cast <目标类型> (表达式)

In [ ]:
#include <iostream>
#include <typeinfo>
using namespace std;

class Base
{
public:
    virtual ~Base(){}
};
class Derived: public Base
{
public:
    void f()
    {
        cout << "Derived f()" << endl;
    }
};
int main(void)
{
    Base *pb, b;
    Derived *pd, d;

    pd = &d;
    pb = pd; //Base * <---------- Derived * 向上造型 隐式转换 编译阶段完成
    pb = dynamic_cast<Base *> (&d); //显示转换

    class C
    {
    public:
        virtual void func(){}
    };
    C c;
    //pb = &c; //error
    pb = dynamic_cast<Base *> (&c);
    if(pb)
    {
        cout << "dynamic_cast ok" << endl;
    }
    else
    {
        cout << "dynamic_cast failed" << endl;
    }
    return 0;
}

向下造型时，动态类型转换会对所需转换的基类指针或引用做检查，如果其目标确实为期望得到的子类类型的对象，则转换成功，否则转换失败

In [ ]:
#include <iostream>
#include <typeinfo>
using namespace std;
class Base
{
public:
    virtual ~Base(){}
};
class Derived: public Base
{
public:
    void f()
    {
        cout << "Derived f()" << endl;
    }
};
int main(void)
{
    Base *pb, b;
    Derived *pd, d;
    pb = &b;
/*
*语法上不报错： 具有父子关系 具有多态特性
*该转换不合理 在运行期间完成转换 转换失败 c++认为不安全
* */
    pd = dynamic_cast<Derived *>(pb);
    if(pd) cout << "ok" << endl;
    else cout << "error" << endl;
    //pd = reinterpret_cast<Derived *>(pb);
    //pd = (Derived *)pb; //C风格强转 可以编译通过
    //pd->f(); //有问题
    pb = &d;
    //向上造型
    pd = dynamic_cast<Derived *>(pb); //编译通过
    if(pd) cout << "ok" << endl;
    else cout << "error" << endl;
    return 0;
}

# 运算符重载

## 什么是运算符重载

使得+ - 等运算符具有新的拓展的功能
同一个运算符可以有不同的功能

例如C++已经对运算符进行了拓展：

<< 既可以作为位运算符，又可以配合cout/cin使用

看下面这个情景：

In [ ]:
#include <iostream>
using namespace std;

class Complex
{
private:
    double r;
    double i;
public:
    Complex(double r, double i)
    {
        this->r = r;
        this->i = i;
    }
    void print(void)
    {
        cout << r << " + " << i << "i" << endl;
    }
};
int main(void)
{
    Complex a(1,2);
    Complex b(3,4);
    a.print();
    b.print();

    // + 运算符可以适用于两个整型数相加
    int x = 10;
    int y = 20;
    int z = x + y;

    //但要想实现用+运算符实现Complex对象相加，需要重载+运算符
    a + b; 
    //想像x+y 实现Complex对象加运算 ＋运算符新的逻辑功能 要实现+运算符的重载
    return 0;
}

## 双目运算符重载


双目运算符：有左右两个操作数的操作符 L#R

算术运算：*、/、%、+、-

关系运算：>、>=、<、<=、==、!=

逻辑运算：&&、||

位运算：&、|、^、<< 、>>

赋值与复合赋值：=、+=、-=、*=、/=、%=、&=、|=、^=、<<= 、>>=

... ...

注意：

表达式结果是右值

左右操作数既可以是左值也可以是右值

实现方式：
+ 成员函数形式：L.operator#(R)
+ 友元函数形式：operator#(L,R)

In [ ]:
#include <iostream>
using namespace std;
class Complex
{
private:
    double r;
    double i;
public:
    Complex(double r, double i)
    {
        this->r = r;
        this->i = i;
    }
    void print(void)
    {
        cout << r << " + " << i << "i" << endl;
    }
    const Complex operator+(const Complex& c)  // 类的成员函数，返回值是const Complex
    {
        Complex tmp(r+c.r, i+c.i);
        return tmp;
    }
    friend const Complex operator-(const Complex& l , const Complex& r);
    //在类中声明友元函数
};
const Complex operator-(const Complex& l, const Complex& r)  //全局函数的形式
{
    Complex tmp(l.r - r.r, l.i - r.i);
    return tmp;
}
int main(void)
{
    Complex a(1,2);
    Complex b(3,4);
    a.print();
    b.print();
    Complex c = a + b;// a.operator+(b);
    c.print();
    Complex d = c - a; // operator-(c, a);
    d.print();
    return 0;
}

可见，运算符重载的本质就是定义一个函数，当用到该运算符时，编译器会自动调用这个函数。

运算符重载是通过函数实现的，本质上是函数重载

`operator运算符名称`这一部分可以看做函数名

还可以使用下面这种更简便的形式

In [ ]:
complex complex::operator+(const complex& A) const
{
    return complex(this->m_real + A.m_real, this->m_image + A.m_image);
    //创建一个临时对象，这个对象没有名称，是一个匿名对象
}